### Àlex Correa Orri 1564967

#### Dataset: https://www.kaggle.com/shaurov/website-classification-using-url

In [1]:
reset()

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
Don't know how to reset  (), please run `%reset?` for details


### MÒDULS I LLIBRERIES

In [2]:
from sklearn.datasets import make_regression
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import scipy.stats
import os
import seaborn as sns
from scipy.stats import shapiro, spearmanr
import math
import sklearn
from sklearn import metrics
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import ipympl
from sklearn.linear_model import LogisticRegression
from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Perceptron
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score, precision_recall_curve, average_precision_score, roc_curve, auc
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import MultinomialNB
%matplotlib widget
import warnings
warnings.filterwarnings('ignore')

### DIRECTORI DE TREBALL

In [3]:
os.getcwd()
path = 'C:/Users/alexc/OneDrive/Escritorio/UNI/3er/Aprenentatge Computacional/PRAC_KAGGLE'
os.chdir(path)
os.getcwd()
# Visualitzarem només 3 decimals per mostra
#pd.set_option('display.float_format', lambda x: '%.3f' % x)

'C:\\Users\\alexc\\OneDrive\\Escritorio\\UNI\\3er\\Aprenentatge Computacional\\PRAC_KAGGLE'

### LECTURA DE DADES

In [4]:
# Carreguem dataset d'exemple
dataset = pd.read_csv('URL_Classification.csv', names=['n', 'url', 'class'], delimiter=',')

In [5]:
dataset=dataset.dropna()

### MODIFICACIÓ DEL DATASET

In [6]:
dataset = dataset.drop(['n'], axis=1)

In [7]:
print(dataset['class'].unique())
len(dataset['class'].unique())

['Adult' 'Arts' 'Business' 'Computers' 'Games' 'Health' 'Home' 'Kids'
 'News' 'Recreation' 'Reference' 'Science' 'Shopping' 'Society' 'Sports']


15

#### Per a fer proves reduim el sample a s dades

In [8]:
s=1000000
dataset1 = dataset.sample(n=s, random_state=10)

In [9]:
dataset1 = dataset

In [10]:
print(dataset1['class'].unique())
len(dataset1['class'].unique())

['Adult' 'Arts' 'Business' 'Computers' 'Games' 'Health' 'Home' 'Kids'
 'News' 'Recreation' 'Reference' 'Science' 'Shopping' 'Society' 'Sports']


15

In [11]:
#dataset1

#### Consultem quines són les paraules més comunes als url del dataset

In [12]:
#dataset1

In [13]:
urls=''
for url in dataset1['url']:
    urls += url
#urls: text concatenat amb tots els strings (sense espais)

In [14]:
vect = CountVectorizer()
mat = vect.fit_transform([urls])
mat
counts = pd.DataFrame(mat.toarray(), columns=vect.get_feature_names())
counts #pandas dataframe amb els valors de les paraules més comunes al string

,00,000,0000,000000,0000000000000000039176http,0000000000000000044918http,00000000002000000003470010http,0000000023http,0000000045http,0000000053ad,...,zzzfelis,zzzgeminiclubsa,zzzlocalsites,zzzptm,zzzsen9ya7c,zzzuzruzglc,zzzv7313hlc,zzzweb,zzzwnppl,zzzzphoecolontradeposts
0,682,69,11,1,1,1,1,1,1,1,...,1,1,3,2,1,1,1,1,1,1


In [15]:
sort_counts = counts.sort_values(by =0, axis=1)
sort_counts = sort_counts[sort_counts.columns[::-1]]

In [16]:
sort_counts #dataframe ordenat per la paraula més comuna

,www,http,com,comhttp,org,htmlhttp,htmhttp,edu,co,uk,...,newyorksurveyrepair,newyorktattoo,newyorkvideoconference,newyorktwistdrill,newyorkultimate,newyorkultrarunning,newyorkun,newyorkunicycle,newyorkvideo,jumpinghfarm
0,1255832,860869,780805,191765,185544,171948,113603,86651,71265,69722,...,1,1,1,1,1,1,1,1,1,1


In [17]:
sort_counts.iloc[:, : 100].columns

Index(['www', 'http', 'com', 'comhttp', 'org', 'htmlhttp', 'htmhttp', 'edu',
       'co', 'uk', 'net', 'geocities', 'index', 'amp', 'orghttp', 'ca',
       'tripod', 'angelfire', 'au', 'ukhttp', 'home', 'ac', 'gov', 'de', 'us',
       'newadvent', 'cathen', 'members', 'nethttp', 'php', 'id', 'en', 'imdb',
       'yahoo', 'html', 'asp', 'shtmlhttp', 'cgi', 'reviews', 'groups',
       'group', 'nl', 'news', 'sports', 'asphttp', 'movies', 'wiki', 'title',
       'wikipedia', 'free', 'phphttp', 'it', 'users', 'library', 'jp', 'info',
       'article', 'freewebs', 'articles', 'name', 'blogspot', 'fr', 'pc',
       'bin', 'cs', 'auhttp', 'gamespot', 'games', 'athletics', 'nz', 'web',
       'cfm', 'english', 'music', 'online', 'pdfhttp', 'tools', 'gamespy',
       'bbc', 'default', 'ign', 'review', 'ietf', 'all', 'main', 'the',
       'sourceforge', 'artists', 'homestead', 'sex', 'history', 'cahttp',
       'objects', 'pages', 'archive', 'aspx', 'soups', 'recipes', 'about',
       'ch'],
   

In [18]:
sort_counts_del = sort_counts
sort_counts_del = sort_counts_del.drop(['www', 'http', 'com', 'comhttp', 'org', 'htmlhttp', 'htmhttp', 'edu', 'co', 'uk', 'net', 'orghttp','html','nethttp','shtmlhttp', 'ukhttp','dehttp','zahttp','jpghttp'], axis=1)

In [19]:
sort_counts_del.iloc[:, : 1000]

,geocities,index,amp,ca,tripod,angelfire,au,home,ac,gov,...,feis,reports,ualberta,fda,pageid,acm,product,utk,bg,ukonline
0,54132,47595,31260,22156,22001,20846,18691,14999,14174,14155,...,323,323,322,322,322,322,321,321,321,320


#### Utilitzarem les ... paraules més comunes (treient els www, http, com, org,...)

In [20]:
words_def = list(sort_counts_del.iloc[:, : 889886].columns)
#for w in words_def:
#    if (('http' in w) or (len(w)<3)):
#        words_def.remove(w)

In [21]:
len(words_def)

889886

In [22]:
#words_def

In [23]:
url_vect = list(dataset1['url'])
vectorizer = CountVectorizer()
fit_vect = vectorizer.fit_transform(url_vect)

#dataset1_word = pd.DataFrame(fit_vect.toarray(), columns=vectorizer.get_feature_names())

In [24]:
X = fit_vect
y = dataset1['class']

In [30]:
x_t, x_v, y_t, y_v = train_test_split(X, y, train_size=0.7)

In [31]:
clf = MultinomialNB()
clf.fit(x_t, y_t)

MultinomialNB()

In [33]:
target_names = np.unique(y_v)
y_pred = clf.predict(x_v)
print(classification_report(y_v, y_pred, target_names=target_names))

              precision    recall  f1-score   support

       Adult       0.99      0.21      0.34     10619
        Arts       0.45      0.62      0.52     76199
    Business       0.31      0.87      0.46     72080
   Computers       0.83      0.20      0.32     35449
       Games       0.77      0.37      0.50     16988
      Health       0.92      0.12      0.21     18020
        Home       0.99      0.21      0.35      8522
        Kids       0.86      0.08      0.15     13718
        News       0.00      0.00      0.00      2627
  Recreation       0.80      0.06      0.11     32051
   Reference       0.73      0.17      0.27     17332
     Science       0.63      0.45      0.53     33078
    Shopping       0.86      0.00      0.01     28547
     Society       0.44      0.59      0.50     73409
      Sports       0.91      0.29      0.43     30254

    accuracy                           0.43    468893
   macro avg       0.70      0.28      0.31    468893
weighted avg       0.62   

In [ ]:
clft = DecisionTreeClassifier(random_state=0, max_depth=100,min_samples_leaf=5,min_samples_split=10)
clft.fit(x_t, y_t)

In [ ]:
target_names = np.unique(y_v)
y_pred = clft.predict(x_v)
print(classification_report(y_v, y_pred, target_names=target_names))

In [ ]:
rfc = RandomForestClassifier(n_estimators=5,criterion='entropy',min_samples_split=5,min_samples_leaf=2,n_jobs=5)
rfc.fit(x_t, y_t)

In [ ]:
y_pred = rfc.predict(x_v)
print(classification_report(y_v, y_pred, target_names=target_names))

In [ ]:
mpl = MLPClassifier()
mpl.fit(x_t, y_t)
round(mpl.score(x_v, y_v),2)